In [1]:
from scipy import constants
import numpy as np

import itertools

import sympy as sy

#import pint
#from pint import UnitRegistry
#ureg = UnitRegistry(auto_reduce_dimensions=True)
#ureg.formatter.default_format = '.4~e'
#
#Q_ = ureg.Quantity
#ureg.define("meter_time = 1*m/c")
#c = ureg.c

In [3]:
def SymLorentzLambda(v = sy.Symbol('v', real=True), all_v=False,axis=1):
	def change_axis(new_axis):
		def change_cols(matrix, new_col):
			matrix_cols = [matrix.col(col) for col in range(matrix.cols)]
			matrix_cols[1], matrix_cols[new_col] = matrix_cols[new_col], matrix_cols[1]
			return sy.Matrix([matrix_cols])

		return change_cols(change_cols(LorentzLambda, axis).T, axis)

	if len(v.name) > 1:
		gamma_name = 'gamma_'+v.name[-1]
	else: gamma_name = 'gamma'
	gamma = sy.Symbol(gamma_name)#1/sy.sqrt(1 - v**2)
	block_1 = sy.Matrix([
		[gamma, -v*gamma],
		[-v*gamma, gamma]
	])

	row_1 = block_1.row_join(sy.zeros(2,2))
	row_2 = sy.zeros(2,2).row_join(sy.diag(1,1))

	LorentzLambda = row_1.col_join(row_2)
	if axis != 1: LorentzLambda = change_axis(axis)
	if all_v: LorentzLambda=LorentzLambda.subs(gamma, 1/sy.sqrt(1-v**2))

	return LorentzLambda

In [4]:
def LorentzLambda(v):
	assert abs(v) < 1, "Speed greater than 1 is invalid."

	gamma = 1/np.sqrt(1 - v**2)
	block_1 = np.array([
		[gamma, -v*gamma],
		[-v*gamma, gamma]
	])

	row_1 = np.hstack((block_1, np.zeros((2,2))))
	row_2 = np.hstack((np.zeros((2,2)), np.eye(2)))

	return np.vstack((row_1, row_2))

In [15]:
class FourVector():
	def __init__(self, components=[0,0,0,0]):
		"""If less than 4 components are provided the array is padded with the necessary zeros."""
		self.components = np.pad(components, (0, 4-len(components)), 'constant', constant_values=0)

	def __str__(self):
		return 'Components: {}, \nNorm: {}\nKind: {}'.format(
				str(list(self.components)), self.norm, self.kind
			)


	def __repr__(self):
		return '4Vector('+str(list(self.components))+')'

	def __rmul__(self, other):
		assert isinstance(other, (int, float)), \
		"unsupported operand type(s) for *: {} and 'FourVector'".format(str(type(other)).strip('<class >'))
		return FourVector(other*self.components)

	def __neg__(self):
		return FourVector(-self.components)

	def __add__(self, other):
		return FourVector(self.components + other.components)

	def __sub__(self, other):
		return self + (-other)

	def __mul__(self, other):
	#	return FourVector(other*self.components)
		assert isinstance(other, FourVector), \
		"unsupported operand type(s) for *: {} and 'FourVector'".format(str(type(other)).strip('<class >'))
		return (np.array([-1, 1, 1, 1]) * self.components * other.components).sum()

	def __getitem__(self, key):
		return self.components[key]


	def transform(self, xVelocity):
		return FourVector(LorentzLambda(xVelocity) @ self.components)

	@property
	def norm(self):
		return self*self

	@property
	def kind(self):
		norm = self.norm
		if norm == 0:
			return 'Null Vector'
		elif norm < 0:
			return 'Time-like Vector'
		else:
			return 'Space-like Vector'

In [30]:
class LorentzTransform():
	def SymLorentzLambda(v = sy.Symbol('v', real=True), axis=1):
		def change_axis(new_axis):
			def change_cols(matrix, new_col):
				matrix_cols = [matrix.col(col) for col in range(matrix.cols)]
				matrix_cols[1], matrix_cols[new_col] = matrix_cols[new_col], matrix_cols[1]
				return sy.Matrix([matrix_cols])

			return change_cols(change_cols(LorentzLambda, axis).T, axis)

		gamma = sy.Symbol('gamma_'+v.name[-1])#1/sy.sqrt(1 - v**2)
		block_1 = sy.Matrix([
			[gamma, -v*gamma],
			[-v*gamma, gamma]
		])

		row_1 = block_1.row_join(sy.zeros(2,2))
		row_2 = sy.zeros(2,2).row_join(sy.diag(1,1))

		LorentzLambda = row_1.col_join(row_2)
		if axis != 1: LorentzLambda = change_axis(axis)
		#if all_v: LorentzLambda=LorentzLambda.subs(gamma, 1/sy.sqrt(1-v**2))

		return LorentzLambda

	def __init__(self, v = sy.Symbol('v', real=True), axis=1):
		self.gamma = 1/sy.sqrt(1-v**2)
		self.matrix = LorentzTransform.SymLorentzLambda(v, axis)

In [16]:
FourVector([1,1,1,1]) - 2*FourVector([-1,1,3,1])

4Vector([np.int64(3), np.int64(-1), np.int64(-5), np.int64(-1)])

In [17]:
FourVector().norm

np.int64(0)

In [18]:
for component in FourVector([-1,1,3,1]):
	print(component)

-1
1
3
1


In [16]:
LorentzLambda(.5)

array([[ 1.15470054, -0.57735027,  0.        ,  0.        ],
       [-0.57735027,  1.15470054,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [22]:
A = FourVector(np.array([-1/2, 1/2]))
print(A)

Components: [np.float64(-0.5), np.float64(0.5), np.float64(0.0), np.float64(0.0)], 
Norm: 0.0
Kind: Null Vector


In [24]:
FourVector([1,1]) * FourVector([1, 1])

np.int64(0)

In [19]:
Vectors = {
	'A': FourVector([5, 1, -1, 0]),
	'B': FourVector([-2, 3, 1, 6]),
	'C': FourVector([2, -2, 0, 0])
}
#list(Vectors.keys())
Vectors_ = dict(
	zip([name + '_' for name in Vectors.keys()],
		[vector.transform(.6) for vector in Vectors.values()])
)
Vectors

{'A': 4Vector([5.0, 1.0, -1.0, 0.0]),
 'B': 4Vector([-2.0, 3.0, 1.0, 6.0]),
 'C': 4Vector([2.0, -2.0, 0.0, 0.0])}

In [20]:
for vector_prod in itertools.combinations_with_replacement(Vectors.keys(), r=2):
	first_vector, second_vector = vector_prod
	print('g({}, {}) = {}'.format(*vector_prod, Vectors[first_vector] * Vectors[second_vector]))

g(A, A) = -23.0
g(A, B) = 12.0
g(A, C) = -12.0
g(B, B) = 42.0
g(B, C) = -2.0
g(C, C) = 0.0


In [21]:
for vector_prod in itertools.combinations_with_replacement(Vectors_.keys(), r=2):
	first_vector, second_vector = vector_prod
	print('g({}, {}) = {}'.format(*vector_prod, Vectors_[first_vector] * Vectors_[second_vector]))

g(A_, A_) = -23.0
g(A_, B_) = 12.0
g(A_, C_) = -12.0
g(B_, B_) = 42.0
g(B_, C_) = -2.0
g(C_, C_) = 0.0


In [22]:
for name, vector in Vectors.items():
	print('{} is a {}'.format(name, vector.kind))

A is a Time-like Vector
B is a Space-like Vector
C is a Null Vector


In [86]:
v_x = sy.Symbol('v^1', real=True)
v_y = sy.Symbol('v^2', real=True)
v_z = sy.Symbol('v^3', real=True)
gamma_v = sy.Symbol('gamma_v', positive=True)
gammas = lambda v: 1/sy.sqrt(1 - v**2)
gamma_x, gamma_y, gamma_z = [gammas(v) for v in [v_x, v_y, v_z]]
compoundTrans = (
	SymLorentzLambda(v_z, axis=3) @ SymLorentzLambda(v_y, axis=2) @ SymLorentzLambda(v_x)
)
#U = sy.Matrix([gamma_x*gamma_y*gamma_z, gamma_x*gamma_y*gamma_z*v_x, gamma_y*gamma_z*v_y, gamma_z*v_z])
compoundTrans#*sy.Matrix([1, v_x, v_y, v_z])

Matrix([
[     gamma_1*gamma_2*gamma_3,    -gamma_1*gamma_2*gamma_3*v^1,    -gamma_2*gamma_3*v^2, -gamma_3*v^3],
[                -gamma_1*v^1,                         gamma_1,                       0,            0],
[        -gamma_1*gamma_2*v^2,         gamma_1*gamma_2*v^1*v^2,                 gamma_2,            0],
[-gamma_1*gamma_2*gamma_3*v^3, gamma_1*gamma_2*gamma_3*v^1*v^3, gamma_2*gamma_3*v^2*v^3,      gamma_3]])

In [24]:
SymLorentzLambda(v_y, axis=2)*sy.Matrix([1, 0, v_y, v_z])

Matrix([
[-gamma_2*v^2**2 + gamma_2],
[                        0],
[                        0],
[                      v^3]])

In [25]:
P = sy.Matrix([300, 299, 0, 0])
sy.simplify(compoundTrans.subs([(v_x, 1/2), (v_y, 1/np.sqrt(3)), (v_z, 1/np.sqrt(2))]) @ P)

Matrix([
[            150.5*gamma_1*gamma_2*gamma_3],
[                            149.0*gamma_1],
[        -86.8912155130387*gamma_1*gamma_2],
[-106.419570568575*gamma_1*gamma_2*gamma_3]])

In [26]:
compoundTrans = SymLorentzLambda(v_z, axis=3, all_v=True) @ SymLorentzLambda(v_y, axis=2, all_v=True) @ SymLorentzLambda(v_x, all_v=True)
sy.simplify(compoundTrans.subs([(v_x, 1/2), (v_y, 1/sy.sqrt(3)), (v_z, 1/sy.sqrt(2))]))

Matrix([
[  1.15470053837925*sqrt(3), -0.577350269189626*sqrt(3),        -1,      -1],
[        -0.577350269189626,           1.15470053837925,         0,       0],
[-0.577350269189626*sqrt(2),  0.288675134594813*sqrt(2), sqrt(6)/2,       0],
[-0.577350269189626*sqrt(6),  0.288675134594813*sqrt(6), sqrt(2)/2, sqrt(2)]])

In [53]:
E = sy.symbols('E^0:4')
B = sy.symbols('B^0:4')

In [54]:
F = sy.Matrix([
	[0, E[1], E[2], E[3]],
	[-E[1], 0, B[3], -B[2]],
	[-E[2], -B[3], 0, B[1]],
	[-E[3], B[2], -B[1], 0]
])
F

Matrix([
[   0,  E^1,  E^2,  E^3],
[-E^1,    0,  B^3, -B^2],
[-E^2, -B^3,    0,  B^1],
[-E^3,  B^2, -B^1,    0]])

In [88]:
v = sy.symbols('v', real=True)
F_ = SymLorentzLambda(v) @ F @ SymLorentzLambda(v)
sy.simplify(F_)

Matrix([
[                      0, E^1*gamma**2*(1 - v**2), gamma*(-B^3*v + E^2),  gamma*(B^2*v + E^3)],
[E^1*gamma**2*(v**2 - 1),                       0,  gamma*(B^3 - E^2*v), gamma*(-B^2 - E^3*v)],
[    gamma*(B^3*v - E^2),    gamma*(-B^3 + E^2*v),                    0,                  B^1],
[   gamma*(-B^2*v - E^3),     gamma*(B^2 + E^3*v),                 -B^1,                    0]])

In [29]:
eta = sy.diag(-1,1,1,1)
Lambda = sy.Matrix([
	[1, -1/2, 0, 0],
	[1, 1/2, 0, 0],
	[0, 0, 1, 0],
	[0, 0, 0, 1]
])
Lambda.T * eta * Lambda

Matrix([
[  0, 1.0, 0, 0],
[1.0,   0, 0, 0],
[  0,   0, 1, 0],
[  0,   0, 0, 1]])